<a href="https://colab.research.google.com/github/saddam-satria/classification/blob/main/Template_ML_Klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Klasifikasi V.01
 
* Digunakan untuk Klasifikasi bukan regresi
*Jalankan baris script yang dibutuhkan saja tidak perlu semua dijalankan
* Template bukan untuk belajar proses machine learning, hanya untuk membantu proses machine learning.


### Import Package 

Wajib di jalankan !

In [ ]:
!pip install jcopml

In [ ]:
#tanda "" bukan script, melainkan penjelasan
#"library sudah disesuaikan dengan machine learning"
 
import pandas as pd #manipulasi data, data table
import numpy as np #reshape array,linear algebra
import seaborn as sns #membuat plot
import matplotlib.pyplot as plt #membuat plot
import os #mengatur isi file direktori komputer (terminal)
from google.colab import files #upload file
import io #membaca file
from sklearn.datasets import load_iris,load_boston
 
 
#"sklearn"
#"library sudah disesuaikan jadi tidak perlu di hapus"
 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,PowerTransformer,PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
 
#"Algoritma, hapus # jika ingin menggunakan algoritma"
 
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier
 
#"Matrix"
#"Opsional library untuk melihat matrix score Dan matrix false - +"
#"Hapus # jika ingin menggunakan"
 
#from sklearn.metrics import accuracy_score,confusion_matrix
 
 
#"jcopml"
#"library yang dibuat oleh JCOp untuk Indonesia" 
#"hapus # jika ingin menggunakannya"
 
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease
from jcopml.plot import plot_confusion_matrix,plot_association_matrix,plot_roc_curve,plot_pr_curve
#from jcopml.tuning import random_search_params as rsp
#from jcopml.tuning import grid_search_params as gsp
 
 
print("-"*2,"Template by Saddam Satria","-"*2,"\n")
print("Library berhasil di import")
print("Selamat Mencoba, Terima Kasih\n")
print("-"*4,"Copy Right ©2020 V.01","-"*4)

###Load Dummy data

Iris dataset

In [ ]:
def iris_dataset():
    dataset = load_iris()
    global df
    df= pd.DataFrame(dataset.data,columns=dataset.feature_names)
    df['jenis bunga']= list(map(lambda x: dataset.target_names[x],dataset.target))
    return df.head()
iris_dataset()

### Load Data

Fungsi load data digunakan untuk memanggil data dari local komputer, data harus berekstensi .csv

In [ ]:
def load_data():
    file = files.upload()
    filename = file.keys()
    for i in (filename):
      global df
      df = pd.read_csv(io.BytesIO(file[i]))
    return df.head()
 
load_data()

### Missing Value

Pengecekan jumlah data yang hilang pada setiap kolom

In [ ]:
def missing_value(df):
  plot_missing_value(df, figsize=(8,8))
  return df.isna().sum()
missing_value(df)

###Drop Columns

 Note:
 
* Fungsi dibuat untuk sekali pakai saja, jika ingin digunakan lagi, hapus nama kolom yang sebelumnya sudah di hapus

In [ ]:
def remove_columns(kolom):
          df.drop(columns=kolom,inplace=True)
          print ("Kolom berhasil di hapus")
          return df.head()
remove_columns(kolom=[''])

###EDA

Plotting

Membuat grafik
 
* Defaultnya adalah 2 baris 2 kolom, jika ingin di ganti, ganti pada script subplots()
 
* Cara menambahkan grafik baru adalah menulis syntax axes[0,0] = diikuti dengan script pemanggilan plot
Ex: Axes[0,0] menunjukan baris ke 1, kolom ke 1

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=False, figsize=(20,14))
fig.suptitle('2 baris 2 kolom')
 
 
axes[0,1]= sns.scatterplot(data=df)

Binning
 
* Membuat rentang pada kolom tertentu, sehingga data numerik menjadi kategori
* Sekali pakai saja, jika ingin yang baru lagi, hapus nama kolom,bins,label sebelumnya dengan yang baru

In [ ]:
df[''] = pd.cut(df[''],bins=[],labels=[])
df.head()

Silahkan analisa data untuk meningkatkan peforma model

### x dan y

* kolom x yang kosong isi dengan nama kolom y
* 
Kolom y yang kosong isi dengan nama kolom y

In [69]:
x = df.drop(columns=[''])
y = df['']

In [ ]:
x.head()

In [ ]:
y.head()

### Splitting Data


* atur ukuruan testing size, train size akan menyesuaikan

In [ ]:
def split_data(x,y,size):
  global xTr,xTs,yTr,yTs
  xTr,xTs,yTr,yTs = train_test_split(x,y, test_size=size/100)
  print("Data Berhasil Di splitting")
  print("Train:",(len(xTr)))
  print("Test:",(len(xTs)))
  return xTr.shape,xTs.shape,yTr.shape,yTs.shape
 
split_data(x,y,size=20)#atur size disni

### Preprocessing pipeline

Scaling data, Encode data kategorik, Dan penentuan algoritma, mengisi data yang hilang menggunakan pipeline

In [ ]:
 
#Numerik pipeline untuk data numerik 
#untuk ganti method scale, tinggal ganti StandardScaler() menjadi scale yang lain
#impute untuk mengisi data yang kosong, terdapat 2 cara yaitu median dan mean, Silahkan ganti 
 
num_pip = Pipeline([
                    ("Impute", SimpleImputer(strategy="mean")),#ganti strategy mean/median
                    ("Scale", StandardScaler())#ganti method scaler
 
                  ])
 
#Kategorik Pipeline untuk data kategorik
 
cat_pip = Pipeline([
                    ("Impute", SimpleImputer(strategy='most_frequent')),
                    ("OneHotEncode", OneHotEncoder())
 
                  ])
 
#penyeleksian kolom berdasarkan tipe kolomnya
#isi bagian [""] untuk menentukan kolom yang ingin digunakan
 
preprocess = ColumnTransformer([
                                ("numerik",num_pip,['']),#isi dengan nama kolom
                                ("kategorik",cat_pip,[''])#isi dengan nama kolom
 
                              ])
 
#Ganti algoritma pada baris model
 
pipeline = Pipeline([
                     ("prep", preprocess),
                     ("model", RandomForestClassifier(n_jobs=-1,verbose=2))#ganti algorithm disini
 
                    ])
 
print("Successful")

### tunning hyperparameter

Jalankan fungsi modeling

In [ ]:
def modeling(xTr,yTr,xTs,yTs,model):
  model.fit(xTr,yTr)
  print(model.best_params_)
  column = [["Best Validation Score", model.best_score_],["Train Score",model.score(xTr,yTr)],["Test Score",model.score(xTs,yTs)]]
  df = pd.DataFrame(column,columns=['Tipe','Hasil'])
  if (model.score(xTr,yTr)) >= 0.95:
    print("Overfitting")
  elif (model.score(xTr,yTr)) <= 0.65:
    print("Underfitting")
  return df
 
print("Fungsi berhasil di jalankan")

Isi parameter yang ingin di iterasi

In [ ]:
parameters = {
   "model__": ,
}

Jalankan tunning (Grid, Random, Robust)
* Ganti pada variabel model, jika ingin menggunakan metode yang lain

In [ ]:
model = RandomizedSearchCV(pipeline, parameters,cv=3,n_iter=50,n_jobs=-1,verbose=1)
modeling(xTr,yTr,xTs,yTs,model)

###Table Hasil Iterasi

Table Hasil percobaan tunning

In [ ]:
result = pd.DataFrame(model.cv_results_)
result.head()

###Plotting Iterasi (Opsional)

Plotting untuk melihat validasi score tiap iterasi

In [ ]:
def plot_cv_results(cv_results, param_x, param_z, metric='mean_test_score'):
    """
    cv_results - cv_results_ attribute of a GridSearchCV instance (or similar)
    param_x - name of grid search parameter to plot on x axis
    param_z - name of grid search parameter to plot by line color
    """
    cv_results = pd.DataFrame(cv_results)
    col_x = 'param_' + param_x
    col_z = 'param_' + param_z
    fig, ax = plt.subplots(1, 1, figsize=(11, 8))
    sns.pointplot(x=col_x, y=metric, hue=col_z, data=cv_results, ci=99, n_boot=64, ax=ax)
    ax.set_title("CV Grid Search Results")
    ax.set_xlabel(param_x)
    ax.set_ylabel(metric)
    ax.legend(title=param_z)
    return fig
plot_cv_results(model.cv_results_,'N_Estimator','Gamma')

### Evaluasi

* Hasil dari pemodelan berupa matrix confusion
* Plot roc, pr opsional, jika ingin menggunakannya hapus #

In [ ]:
def plot_klasifikasi(xTr,yTr,xTs,yTs,model):
    plot_confusion_matrix(xTr,yTr,xTs,yTs,model)
    #plot_pr_curve(xTr,yTr,xTs,yTs,model)
    #plot_roc_curve(xTr,yTr,xTs,yTs,model)
plot_klasifikasi(xTr,yTr,xTs,yTs,model)

#### Feature Importance

Melihat kolom yang mempengaruhi peforma model

In [ ]:
df_imp = mean_score_decrease(xTr,yTr,model,plot=True,topk=10)